In [1]:
import tensorflow as tf
from bert import modeling
import os
import create_input
import tokenization
import numpy as np
import pandas as pd
import random

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


需要下载Tensorflow Bert源码和Pretrain的BERT-Base-Chinese checkpoint

In [2]:
# 这里是下载下来的bert配置文件
bert_config = modeling.BertConfig.from_json_file("chinese_L-12_H-768_A-12/bert_config.json")
vocab_file="chinese_L-12_H-768_A-12/vocab.txt"
batch_size=20
num_labels=2# 类别数量
is_training=True
max_seq_length=64
iter_num=2
lr=0.00005
if max_seq_length > bert_config.max_position_embeddings: # 模型有个最大的输入长度 512
    raise ValueError("超出模型最大长度")

data_df = pd.read_csv('D:\\tensorflow\\competition\\train_data.csv')

texts=data_df['comment'].values.tolist()
labels=data_df['label'].values.tolist()
del data_df
text_label = [(texts[i],labels[i]) for i in range(len(labels))]
random.seed(42)
random.shuffle(text_label)

在本机跑的，训练测试数据条数多了就内存溢出了，这里训练每个batch 20条数据，验证数据 40条，数据够多没有使用交叉验证。

In [10]:
x_val_texts=[]
y_val_labels=[]
x_train_texts=[]
y_train_labels=[]
for i in range(len(text_label)):
    if i<40:
        x_val_texts.append(text_label[i][0])
        y_val_labels.append(text_label[i][1])
    else:
        x_train_texts.append(text_label[i][0])
        y_train_labels.append(text_label[i][1])
del texts
del labels
del text_label

In [12]:
def get_bert_param_lists(texts):
    tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file) # token 处理器，主要作用就是 分字，将字转换成ID。vocab_file 字典文件路径
    input_idsList=[]
    input_masksList=[]
    segment_idsList=[]
    for t in texts:
        single_input_id, single_input_mask, single_segment_id=create_input.convert_single_example(max_seq_length,tokenizer,t)
        input_idsList.append(single_input_id)
        input_masksList.append(single_input_mask)
        segment_idsList.append(single_segment_id)
    input_idsArray = np.asarray(input_idsList,dtype=np.int32)
    input_masksArray = np.asarray(input_masksList,dtype=np.int32)
    segment_idsArray = np.asarray(segment_idsList,dtype=np.int32)
    return input_idsArray,input_masksArray,segment_idsArray

将数据转换成Bert能够使用的格式

input_idsList：根据BERT-Base-Chinese checkpoint中的vocabtxt中每个字出现的index，将训练文本中的每一个字替换为vocab.txt中的index，需要添加开始CLS和结束SEP

input_masksList：包含开始CLS和结束SEP有字就填1

segment_idsList：seq2seq类任务同时传入两句训练关联训练数据时，有意义，传入一句训练数据则都为0

以上三个list需要用0补齐到max_seq_length的长度

train_idsList,train_masksList,train_segment_idsList = get_bert_param_lists(x_train_texts)
train_labels=np.asarray(y_train_labels,dtype=np.float32)

In [14]:
train_labels=train_labels.reshape(-1,1)

In [15]:
train_labels.shape

(44028, 1)

In [16]:
tf.reset_default_graph()

input_ids=tf.placeholder(shape=[None,max_seq_length],dtype=tf.int32,name="input_ids")
input_mask=tf.placeholder(shape=[None,max_seq_length],dtype=tf.int32,name="input_mask")
segment_ids=tf.placeholder(shape=[None,max_seq_length],dtype=tf.int32,name="segment_ids")

input_labels=tf.placeholder (shape=[None,1],dtype=tf.float32,name="input_ids")
is_training=tf.placeholder_with_default(True,shape=(),name='is_training')

In [19]:
# 创建bert模型
with tf.name_scope('Bert'):
    model = modeling.BertModel(
        config=bert_config,
        is_training=True,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=False # 这里如果使用TPU 设置为True，速度会快些。使用CPU 或GPU 设置为False ，速度会快些。
    )
    # output_layer = model.get_sequence_output()# 这个获取每个token的output 输入数据[batch_size, seq_length, embedding_size] 如果做seq2seq 或者ner 用这个
    output_layer = model.get_pooled_output() # 这个获取句子的output
    hidden_size = output_layer.shape[-1].value #获取输出的维度

In [20]:
# 后面就简单了，就是一个全连接
with tf.name_scope('Output'):
    output_layer = tf.layers.dropout(output_layer, 0.5,training=is_training)
with tf.name_scope('Loss'):
    logits = tf.layers.dense(output_layer,1,name="logits")
    sigmoid_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=input_labels, name="sigmoid_loss")
    loss = tf.reduce_mean(sigmoid_loss, name="loss")
with tf.name_scope('Predict'):
    one = tf.ones_like(logits)
    zero = tf.zeros_like(logits)
    predict = tf.where(logits <0.5, x=zero, y=one,name='prediction')
with tf.name_scope('Eval'):
    acc = tf.reduce_mean(tf.cast(tf.equal(input_labels, tf.cast(predict, dtype=tf.float32)), "float"), name="accuracy")
with tf.name_scope('Train'):
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

In [21]:
#bert模型参数初始化的地方
init_checkpoint = "chinese_L-12_H-768_A-12/bert_model.ckpt"
use_tpu = False
# 获取模型中所有的训练参数。
tvars = tf.trainable_variables()
# 加载BERT模型
(assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars,
                                                                                       init_checkpoint)
tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/beta:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/embeddings/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/gamma:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/embeddings/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/embeddings/position_embeddings:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/embeddings/position_embeddings
INFO:tensorflow:Initialize variable bert/embeddings/token_type_embeddings:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/embeddings/token_type_embeddings
INFO:tensorflow:Initialize variable bert/embeddings/word_embeddings:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/embeddings/word_embeddings
INFO:tensorflow:Initialize variable bert/encoder/layer_0/attention/output/LayerNorm/beta:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/la

INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/query/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_10/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/query/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_10/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/value/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_10/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/value/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_10/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/intermediate/dense/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_10/intermediate/dense/bias
INFO:tensorflow:Initi

INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/output/LayerNorm/gamma:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_3/attention/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/output/dense/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_3/attention/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/output/dense/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_3/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/key/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_3/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/key/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_3/attention/self/key/kernel
INFO:tensorfl

INFO:tensorflow:Initialize variable bert/encoder/layer_5/output/LayerNorm/beta:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_5/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_5/output/LayerNorm/gamma:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_5/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_5/output/dense/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_5/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_5/output/dense/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_5/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/output/LayerNorm/beta:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_6/attention/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attentio

INFO:tensorflow:Initialize variable bert/encoder/layer_8/attention/self/query/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_8/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_8/attention/self/value/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_8/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_8/attention/self/value/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_8/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_8/intermediate/dense/bias:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_8/intermediate/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_8/intermediate/dense/kernel:0 from checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt with bert/encoder/layer_8/intermediate/dense/kernel
INFO:tensorflow:Initialize vari

In [22]:
def shuffle_batch(labels,idsList,masksList,segmentList,batch_size):
    index = np.random.permutation(len(labels))
    n_batches = len(labels) // batch_size
    for batch_index in np.array_split(index,n_batches):
        batch_labels,batch_input_idsList,batch_input_masksList,batch_segment_idsList=labels[batch_index],idsList[batch_index],masksList[batch_index],segmentList[batch_index]
        yield batch_labels,batch_input_idsList,batch_input_masksList,batch_segment_idsList

In [23]:
val_idsList,val_masksList,val_segment_idsList = get_bert_param_lists(x_val_texts)
val_labels=np.asarray(y_val_labels,dtype=np.float32).reshape(-1,1)

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(iter_num):
    for batch_labels,batch_input_idsList,batch_input_masksList,batch_segment_idsList in shuffle_batch(train_labels,train_idsList,train_masksList,train_segment_idsList,batch_size):
        sess.run(train_op,feed_dict={
            input_ids:batch_input_idsList,input_mask:batch_input_masksList,
            segment_ids:batch_segment_idsList,input_labels:batch_labels,is_training:True})
        l,a = sess.run([loss,acc],feed_dict={input_ids:val_idsList,input_mask:val_masksList,
            segment_ids:val_segment_idsList,input_labels:val_labels,is_training:False})
        print("epoch:{},准确率:{},损失函数:{}".format(i,a,l))

epoch:0,准确率:0.6499999761581421,损失函数:0.6959041953086853
epoch:0,准确率:0.6499999761581421,损失函数:0.6188446283340454
epoch:0,准确率:0.574999988079071,损失函数:0.6139447093009949
epoch:0,准确率:0.44999998807907104,损失函数:0.6296599507331848
epoch:0,准确率:0.574999988079071,损失函数:0.579159140586853
epoch:0,准确率:0.7749999761581421,损失函数:0.568206787109375
epoch:0,准确率:0.7250000238418579,损失函数:0.5275169610977173
epoch:0,准确率:0.6000000238418579,损失函数:0.5670086145401001
epoch:0,准确率:0.8500000238418579,损失函数:0.5533908009529114
epoch:0,准确率:0.824999988079071,损失函数:0.5285813212394714
epoch:0,准确率:0.75,损失函数:0.5584396719932556
epoch:0,准确率:0.6499999761581421,损失函数:0.5369657874107361
epoch:0,准确率:0.699999988079071,损失函数:0.5719677805900574
epoch:0,准确率:0.550000011920929,损失函数:0.5536437034606934
epoch:0,准确率:0.800000011920929,损失函数:0.5195783376693726
epoch:0,准确率:0.75,损失函数:0.5121825337409973
epoch:0,准确率:0.7250000238418579,损失函数:0.5079748630523682
epoch:0,准确率:0.6499999761581421,损失函数:0.5389186143875122
epoch:0,准确率:0.44999998807907104,损失函数:0.632513

epoch:0,准确率:0.7749999761581421,损失函数:0.4534526467323303
epoch:0,准确率:0.800000011920929,损失函数:0.438393771648407
epoch:0,准确率:0.800000011920929,损失函数:0.3904232084751129
epoch:0,准确率:0.7749999761581421,损失函数:0.4076159596443176
epoch:0,准确率:0.625,损失函数:0.4970037341117859
epoch:0,准确率:0.6000000238418579,损失函数:0.5575205087661743
epoch:0,准确率:0.550000011920929,损失函数:0.5731350779533386
epoch:0,准确率:0.6000000238418579,损失函数:0.5326361656188965
epoch:0,准确率:0.5249999761581421,损失函数:0.5411931276321411
epoch:0,准确率:0.6499999761581421,损失函数:0.5004919171333313
epoch:0,准确率:0.7749999761581421,损失函数:0.4565744996070862
epoch:0,准确率:0.875,损失函数:0.3878578543663025
epoch:0,准确率:0.824999988079071,损失函数:0.4308401048183441
epoch:0,准确率:0.824999988079071,损失函数:0.40611928701400757
epoch:0,准确率:0.8500000238418579,损失函数:0.40120917558670044
epoch:0,准确率:0.824999988079071,损失函数:0.3969985544681549
epoch:0,准确率:0.824999988079071,损失函数:0.3797772228717804
epoch:0,准确率:0.800000011920929,损失函数:0.3750211298465729
epoch:0,准确率:0.875,损失函数:0.35718756914138794


epoch:0,准确率:0.824999988079071,损失函数:0.3498939871788025
epoch:0,准确率:0.824999988079071,损失函数:0.32983627915382385
epoch:0,准确率:0.8500000238418579,损失函数:0.35567042231559753
epoch:0,准确率:0.8500000238418579,损失函数:0.34208595752716064
epoch:0,准确率:0.800000011920929,损失函数:0.3651476800441742
epoch:0,准确率:0.800000011920929,损失函数:0.3592322766780853
epoch:0,准确率:0.8500000238418579,损失函数:0.3504372537136078
epoch:0,准确率:0.8500000238418579,损失函数:0.3778960108757019
epoch:0,准确率:0.800000011920929,损失函数:0.3874616026878357
epoch:0,准确率:0.800000011920929,损失函数:0.41590699553489685
epoch:0,准确率:0.824999988079071,损失函数:0.3936966359615326
epoch:0,准确率:0.824999988079071,损失函数:0.36911147832870483
epoch:0,准确率:0.800000011920929,损失函数:0.38242095708847046
epoch:0,准确率:0.7749999761581421,损失函数:0.4226403832435608
epoch:0,准确率:0.800000011920929,损失函数:0.42834988236427307
epoch:0,准确率:0.800000011920929,损失函数:0.4664901793003082
epoch:0,准确率:0.7749999761581421,损失函数:0.36876899003982544
epoch:0,准确率:0.824999988079071,损失函数:0.3979092836380005
epoch:0,准确率:0.

epoch:0,准确率:0.824999988079071,损失函数:0.3683646619319916
epoch:0,准确率:0.7749999761581421,损失函数:0.37421661615371704
epoch:0,准确率:0.8500000238418579,损失函数:0.3572896122932434
epoch:0,准确率:0.875,损失函数:0.34285563230514526
epoch:0,准确率:0.875,损失函数:0.3421761393547058
epoch:0,准确率:0.875,损失函数:0.35229748487472534
epoch:0,准确率:0.824999988079071,损失函数:0.39102742075920105
epoch:0,准确率:0.800000011920929,损失函数:0.44718098640441895
epoch:0,准确率:0.800000011920929,损失函数:0.4489615559577942
epoch:0,准确率:0.75,损失函数:0.4625968933105469
epoch:0,准确率:0.824999988079071,损失函数:0.43130865693092346
epoch:0,准确率:0.8500000238418579,损失函数:0.3775840699672699
epoch:0,准确率:0.875,损失函数:0.34274229407310486
epoch:0,准确率:0.8500000238418579,损失函数:0.3621511459350586
epoch:0,准确率:0.800000011920929,损失函数:0.35804373025894165
epoch:0,准确率:0.824999988079071,损失函数:0.3400666415691376
epoch:0,准确率:0.8999999761581421,损失函数:0.2920394539833069
epoch:0,准确率:0.875,损失函数:0.3106486201286316
epoch:0,准确率:0.925000011920929,损失函数:0.29563650488853455
epoch:0,准确率:0.8999999761581421,损失

epoch:0,准确率:0.824999988079071,损失函数:0.3576021194458008
epoch:0,准确率:0.8500000238418579,损失函数:0.3238697946071625
epoch:0,准确率:0.8500000238418579,损失函数:0.3670702576637268
epoch:0,准确率:0.8500000238418579,损失函数:0.3276194632053375
epoch:0,准确率:0.875,损失函数:0.3712844252586365
epoch:0,准确率:0.875,损失函数:0.3502817749977112
epoch:0,准确率:0.824999988079071,损失函数:0.4014585018157959
epoch:0,准确率:0.824999988079071,损失函数:0.41487598419189453
epoch:0,准确率:0.800000011920929,损失函数:0.42857521772384644
epoch:0,准确率:0.824999988079071,损失函数:0.39927008748054504
epoch:0,准确率:0.824999988079071,损失函数:0.4170863628387451
epoch:0,准确率:0.824999988079071,损失函数:0.4005574584007263
epoch:0,准确率:0.875,损失函数:0.34109288454055786
epoch:0,准确率:0.875,损失函数:0.40080779790878296
epoch:0,准确率:0.8999999761581421,损失函数:0.3300950527191162
epoch:0,准确率:0.8500000238418579,损失函数:0.36173155903816223
epoch:0,准确率:0.8500000238418579,损失函数:0.32299765944480896
epoch:0,准确率:0.800000011920929,损失函数:0.37739014625549316
epoch:0,准确率:0.800000011920929,损失函数:0.4380766749382019
epoch:0,

...

epoch:1,准确率:0.8999999761581421,损失函数:0.2988259494304657
epoch:1,准确率:0.875,损失函数:0.2857884466648102
epoch:1,准确率:0.875,损失函数:0.30981355905532837
epoch:1,准确率:0.8500000238418579,损失函数:0.29560914635658264
epoch:1,准确率:0.8999999761581421,损失函数:0.2513038218021393
epoch:1,准确率:0.8999999761581421,损失函数:0.3009490370750427
epoch:1,准确率:0.875,损失函数:0.26486626267433167
epoch:1,准确率:0.8999999761581421,损失函数:0.3130597174167633
epoch:1,准确率:0.824999988079071,损失函数:0.32712310552597046
epoch:1,准确率:0.875,损失函数:0.31987422704696655
epoch:1,准确率:0.875,损失函数:0.29221728444099426
epoch:1,准确率:0.8500000238418579,损失函数:0.31347572803497314
epoch:1,准确率:0.8500000238418579,损失函数:0.29339057207107544
epoch:1,准确率:0.8500000238418579,损失函数:0.33863481879234314
epoch:1,准确率:0.8999999761581421,损失函数:0.26927605271339417
epoch:1,准确率:0.875,损失函数:0.2977745831012726
epoch:1,准确率:0.8500000238418579,损失函数:0.3385165333747864
epoch:1,准确率:0.875,损失函数:0.31571733951568604
epoch:1,准确率:0.875,损失函数:0.3177326023578644
epoch:1,准确率:0.8500000238418579,损失函数:0.32584029436

epoch:1,准确率:0.875,损失函数:0.39001286029815674
epoch:1,准确率:0.800000011920929,损失函数:0.46355152130126953
epoch:1,准确率:0.8500000238418579,损失函数:0.42176467180252075
epoch:1,准确率:0.824999988079071,损失函数:0.4011288583278656
epoch:1,准确率:0.875,损失函数:0.37097474932670593
epoch:1,准确率:0.875,损失函数:0.4060961604118347
epoch:1,准确率:0.824999988079071,损失函数:0.42669981718063354
epoch:1,准确率:0.8500000238418579,损失函数:0.3542010188102722
epoch:1,准确率:0.8500000238418579,损失函数:0.4075586199760437
epoch:1,准确率:0.875,损失函数:0.38589876890182495
epoch:1,准确率:0.8500000238418579,损失函数:0.3455398380756378
epoch:1,准确率:0.8500000238418579,损失函数:0.3761550784111023
epoch:1,准确率:0.824999988079071,损失函数:0.426140695810318
epoch:1,准确率:0.8500000238418579,损失函数:0.3952856659889221
epoch:1,准确率:0.824999988079071,损失函数:0.4489723742008209
epoch:1,准确率:0.8500000238418579,损失函数:0.4472745358943939
epoch:1,准确率:0.824999988079071,损失函数:0.4723624289035797
epoch:1,准确率:0.800000011920929,损失函数:0.4786617159843445
epoch:1,准确率:0.800000011920929,损失函数:0.38318949937820435
epoch:1,准

epoch:1,准确率:0.875,损失函数:0.3677801489830017
epoch:1,准确率:0.8999999761581421,损失函数:0.3535751700401306
epoch:1,准确率:0.875,损失函数:0.3652805685997009
epoch:1,准确率:0.875,损失函数:0.4074937701225281
epoch:1,准确率:0.875,损失函数:0.3934052288532257
epoch:1,准确率:0.8500000238418579,损失函数:0.3810460865497589
epoch:1,准确率:0.925000011920929,损失函数:0.3222036361694336
epoch:1,准确率:0.875,损失函数:0.43044596910476685
epoch:1,准确率:0.824999988079071,损失函数:0.4380396902561188
epoch:1,准确率:0.8500000238418579,损失函数:0.3372573256492615
epoch:1,准确率:0.8500000238418579,损失函数:0.3845652937889099
epoch:1,准确率:0.8500000238418579,损失函数:0.3994569182395935
epoch:1,准确率:0.800000011920929,损失函数:0.40812087059020996
epoch:1,准确率:0.7749999761581421,损失函数:0.5053143501281738
epoch:1,准确率:0.75,损失函数:0.5059276819229126
epoch:1,准确率:0.75,损失函数:0.6028772592544556
epoch:1,准确率:0.75,损失函数:0.5384858250617981
epoch:1,准确率:0.75,损失函数:0.6372555494308472
epoch:1,准确率:0.800000011920929,损失函数:0.5125778317451477
epoch:1,准确率:0.824999988079071,损失函数:0.490116685628891
epoch:1,准确率:0.82499998807

epoch:1,准确率:0.875,损失函数:0.34868454933166504
epoch:1,准确率:0.875,损失函数:0.2950877547264099
epoch:1,准确率:0.875,损失函数:0.3983026146888733
epoch:1,准确率:0.824999988079071,损失函数:0.41637760400772095
epoch:1,准确率:0.8500000238418579,损失函数:0.399386465549469
epoch:1,准确率:0.8500000238418579,损失函数:0.37882018089294434
epoch:1,准确率:0.875,损失函数:0.39590784907341003
epoch:1,准确率:0.8500000238418579,损失函数:0.43788856267929077
epoch:1,准确率:0.8500000238418579,损失函数:0.2989388406276703
epoch:1,准确率:0.8500000238418579,损失函数:0.36447426676750183
epoch:1,准确率:0.8500000238418579,损失函数:0.4056434631347656
epoch:1,准确率:0.8500000238418579,损失函数:0.4454188346862793
epoch:1,准确率:0.8500000238418579,损失函数:0.35760602355003357
epoch:1,准确率:0.8500000238418579,损失函数:0.41329678893089294
epoch:1,准确率:0.8500000238418579,损失函数:0.3970552384853363
epoch:1,准确率:0.824999988079071,损失函数:0.38815364241600037
epoch:1,准确率:0.824999988079071,损失函数:0.3888463079929352
epoch:1,准确率:0.875,损失函数:0.3701556324958801
epoch:1,准确率:0.875,损失函数:0.3379600942134857
epoch:1,准确率:0.875,损失函数:0.358

epoch:1,准确率:0.8500000238418579,损失函数:0.4548102021217346
epoch:1,准确率:0.8500000238418579,损失函数:0.4664227366447449
epoch:1,准确率:0.824999988079071,损失函数:0.4909513592720032
epoch:1,准确率:0.875,损失函数:0.43444520235061646
epoch:1,准确率:0.875,损失函数:0.4335673749446869
epoch:1,准确率:0.8500000238418579,损失函数:0.4601937234401703
epoch:1,准确率:0.8500000238418579,损失函数:0.4360676407814026
epoch:1,准确率:0.824999988079071,损失函数:0.4514245092868805
epoch:1,准确率:0.824999988079071,损失函数:0.47452059388160706
epoch:1,准确率:0.8500000238418579,损失函数:0.4801939129829407
epoch:1,准确率:0.8500000238418579,损失函数:0.4900158941745758
epoch:1,准确率:0.824999988079071,损失函数:0.46681898832321167
epoch:1,准确率:0.8500000238418579,损失函数:0.4565238356590271
epoch:1,准确率:0.8500000238418579,损失函数:0.4178715646266937
epoch:1,准确率:0.7749999761581421,损失函数:0.5084296464920044
epoch:1,准确率:0.8500000238418579,损失函数:0.4314613938331604
epoch:1,准确率:0.824999988079071,损失函数:0.4487614631652832
epoch:1,准确率:0.824999988079071,损失函数:0.43791142106056213
epoch:1,准确率:0.824999988079071,损失函数:0.4

In [25]:
test_data_df = pd.read_csv('D:\\tensorflow\\competition\\test_data.csv')
texts_test=test_data_df['comment'].values.tolist()

,comment
0,画面很美但台词实在是太太太矫情不是十几岁少年的正常青涩哪怕故作的沧桑而像没文化的油腻中年对键...
1,这片子好看好看好看好看重要的问题问四遍竟然是前百失望致幻的嗨药是弱者逃避现实的选择你觉得现实...
2,老电影有老电影说不出来的美感即使是近三个小时的电影细节做的仍非精致七武士的故事人类百科
3,拯救世人的疾病和灾难对人世间的所有痛苦感同身受最后却遭受孤独误解蒙冤走上电椅十字架这其实是在...
4,非常非常反感人与机器之间所谓的感情很傻


In [27]:
del test_data_df

In [28]:
test_idsList,test_masksList,test_segment_idsList = get_bert_param_lists(texts_test)
index_test = np.arange(len(texts_test))
n_batches_test = len(texts_test) // batch_size
y_pred=[]
for bidx in np.array_split(index_test,n_batches_test):
    batch_test_idsList,batch_test_masksList,batch_test_segment_idsList=test_idsList[bidx],test_masksList[bidx],test_segment_idsList[bidx]
    y_pred_batch = sess.run(predict,feed_dict={
            input_ids:batch_test_idsList,input_mask:batch_test_masksList,
            segment_ids:batch_test_segment_idsList,is_training:False})
    y_pred.extend(y_pred_batch)

result = [int(a.tolist()[0]) for a in y_pred]

with open('result1.csv','w') as f1:
    for item in result:
        f1.write(str(item)+'\n')

sess.close()